In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data = pd.read_csv("./use_this_data.csv")

In [35]:
vdw_vol = {     # Values from Proteins: Structures and Molecular Properties by Thomas Creighton
    "A": 67,
    "R": 148,
    "N": 96,
    "D": 91,
    "C": 86,
    "Q": 114,
    "E": 109,
    "G": 48,
    "H": 118,
    "I": 124,
    "L": 124,
    "K": 135,
    "M": 124,
    "F": 135,
    "P": 90,
    "S": 90,
    "T": 93,
    "W": 163,
    "Y": 141,
    "V": 105
}
vdw_vol = dict(sorted(vdw_vol.items(), key=lambda x:x[1]))

s_to_l = []
not_contains_ala = []

# NOTE: I am counting any datapoint that has ONE small-to-large mutation or ONE mutation to alanine
for _, point in data.iterrows():
    muts = re.split(";", point["Mutations"])
    tracker_ala = False
    tracker = False
    for mut in muts:
        aas = re.split(":", re.sub(
            r"(\w):(\w)(\d+)\w*(\w)", r"\2:\4", mut))
        if aas[1] == 'A':
            tracker_ala = True
        if vdw_vol[aas[0]] < vdw_vol[aas[1]]:
            tracker = True
    s_to_l.append(tracker)
    not_contains_ala.append(not tracker_ala)

s_to_l = pd.Series(s_to_l)
print(s_to_l.head())

single_ala = (data.loc[:, "LD"] == 1) & (data.loc[:, "Mutations"].str.endswith("A"))
print(single_ala.head())
multiple_mut = data.loc[:, "LD"] > 1
print(multiple_mut.head())
multiple_mut_no_ala = (multiple_mut) & not_contains_ala
print(multiple_mut_no_ala.head())
data["Single_mut_A"] = single_ala
data["Small_to_large"] = s_to_l
data["Multiple_mut"] = multiple_mut
data["Multiple_mut_no_A"] = multiple_mut_no_ala

print(data.head())

0    False
1    False
2    False
3    False
4    False
dtype: bool
0    True
1    True
2    True
3    True
4    True
dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: LD, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: LD, dtype: bool
   #PDB Mutations  ddG(kcal/mol)   Source  LD  Jump  Interface?  Single_mut_A  \
0  1DQJ    H:D32A            2.0  AB-Bind   1     1        True          True   
1  1DQJ    H:W98A            4.9  AB-Bind   1     1        True          True   
2  1DQJ    H:Y33A            5.5  AB-Bind   1     1        True          True   
3  1DQJ    H:Y50A            6.9  AB-Bind   1     1        True          True   
4  1DQJ    H:Y53A            1.2  AB-Bind   1     1        True          True   

   Small_to_large  Multiple_mut  Multiple_mut_no_A  
0           False         False              False  
1           False         False              False  
2           False         False              False  
3           False 

In [ ]:
data.to_csv("./use_this_data.csv", index=False)